In [1]:
from IPython.display import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

2021-11-24 19:28:46.016795: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
from tensorflow.python.client import device_lib
gpus = tf.config.experimental.list_physical_devices('GPU')
print('='*100)
print(gpus)
print('='*100)
print("GPU Available: ", tf.test.is_gpu_available())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available:  True


2021-11-24 19:28:46.888907: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-11-24 19:28:46.916505: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.759GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-11-24 19:28:46.917538: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:0b:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.759GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-11-24 19:28:46.917565: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-11-24 19:28:46.919146: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcubl

In [3]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2021-11-24 19:28:49.079614: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.759GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-11-24 19:28:49.080769: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:0b:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.759GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-11-24 19:28:49.080837: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-11-24 19:28:49.080891: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-11-24 19:28:49.080923: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libc

In [ ]:
TRAINING_DIR = "/root/data/cut_data_limit10000/train/"
VALIDATION_DIR = "/root/data/cut_data_limit10000/validation/"
TEST_DIR = "/root/data/cut_data_limit10000/test/"

In [ ]:
training_datagen = ImageDataGenerator(
                                        rescale=1. / 255,
                                        rotation_range=20,
                                        width_shift_range=0.1,
                                        height_shift_range=0.1,
                                        zoom_range=0.05,
                                        horizontal_flip=True,
                                        fill_mode='reflect', 
                                        validation_split=0.2
                                        )

In [ ]:
training_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                          batch_size=32, 
                                                          target_size=(150, 150), 
                                                          class_mode='binary',
                                                          shuffle=True 
                                                        #   subset='training',
                                                         )

In [ ]:
test_datagen = ImageDataGenerator(
                    rescale = 1/255
                )

In [ ]:
validation_generator = test_datagen.flow_from_directory(VALIDATION_DIR, 
                                                          batch_size=32, 
                                                          target_size=(150, 150), 
                                                          class_mode='binary'
                                                        #   subset='validation', 
                                                         )

In [ ]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [ ]:
base_model.trainable = False

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))
print(inputs)
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be normalized
# from (0, 255) to a range (-1., +1.), the normalization layer
# does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()

mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(x)
# norm_layer.set_weights([mean, var])

In [ ]:
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# Regularize with dropout
x = keras.layers.Dropout(0.2)(x)  
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)
checkpoint_path = f"/root/data/model/Xception_model/tmp_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                            save_weights_only=True, 
                            save_best_only=True, 
                            monitor='val_loss', 
                            verbose=1)
epochs = 5
model.fit(training_generator,
          epochs=epochs,
          validation_data=validation_generator,
          callbacks=[checkpoint])

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
model.save(f"/root/data/model/Xception_model/1113_Xception_model.h5")

In [ ]:
test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                 batch_size=32, 
                                                 target_size=(150, 150), 
                                                 class_mode='binary'
                                                 #   subset='validation', 
                                                )

In [ ]:
model.evaluate(test_generator)

In [ ]:
import cv2
img = cv2.imread('/root/final_project/images2.png')

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
img = cv2.resize(img, dsize = (150,150))

In [ ]:
img = img.reshape(-1, 150, 150, 3)

In [ ]:
model.predict(img)